In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import sys
sys.path.append('../')

import numpy as onp
import jax
import jax.numpy as np

from jax import lax, random
from jax.api import grad, jit, vmap
from jax.config import config
from jax.experimental import optimizers
from jax.experimental.stax import logsoftmax

config.update('jax_enable_x64', True)

from neural_tangents import stax

from functools import partial

# Attacking
from cleverhans.utils import clip_eta, one_hot

# Plotting
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils import *

sns.set_style(style='white')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

"""
diag_reg:
    a scalar representing the strength of the diagonal regularization for
    `k_train_train`, i.e. computing `k_train_train + diag_reg * I` during
    Cholesky factorization or eigendecomposition.
"""
diag_reg = 1e-5

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

# data

In [2]:
# [1, 4, 16, 64, 256, 1024, 4096, 16384, 65536]

In [3]:
DATASET = 'cifar10'
class_num   = 10
image_shape = None

train_size = 4096
test_size = 64
test_batch_size  = 16
time = 65536
eps = 0.1

if DATASET =='mnist':
    image_shape = (28, 28)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_test = x_train.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

# model

In [8]:
def correct(mean, ys):
    return onp.argmax(mean, axis=-1) == onp.argmax(ys, axis=-1)

In [9]:
def ConvBlock(channels, W_std, b_std, strides=(1,1)):
    return stax.serial(stax.Conv(out_chan=channels, filter_shape=(3,3), strides=strides, padding='SAME',
                                 W_std=W_std, b_std=b_std), 
                       stax.Relu(do_backprop=True))

def ConvGroup(n, channels, stride, W_std, b_std, last_stride=False):
    blocks = []
    if last_stride:
        for i in range(n-1):
            blocks += [ConvBlock(channels, W_std, b_std, stride)]
        blocks += [ConvBlock(channels, W_std, b_std, (2, 2))]
    
    else:
        for i in range(n):
            blocks += [ConvBlock(channels, W_std, b_std, stride)]
        
    return stax.serial(*blocks)
        
def VGG19_stride(class_num=class_num):
    
    return stax.serial(
        ConvGroup(n=2, channels=64 , stride=(1,1), W_std=1.76, b_std=0.18, last_stride=True),
        ConvGroup(n=2, channels=128, stride=(1,1), W_std=1.76, b_std=0.18, last_stride=True),
        ConvGroup(n=4, channels=256, stride=(1,1), W_std=1.76, b_std=0.18, last_stride=True),
        ConvGroup(n=4, channels=512, stride=(1,1), W_std=1.76, b_std=0.18, last_stride=True),
        ConvGroup(n=4, channels=512, stride=(1,1), W_std=1.76, b_std=0.18, last_stride=True),
        stax.Flatten(),
        stax.Dense(4096), stax.Relu(do_backprop=True),
        stax.Dense(4096), stax.Relu(do_backprop=True),
        stax.Dense(class_num))

In [10]:
init_fn, apply_fn, kernel_fn = VGG19_stride(class_num)

In [11]:
batch_kernel_fn = nt.batch(kernel_fn, batch_size=256, store_on_device=False)

In [12]:
# kernel_train_m = batch_kernel_fn(x_train, None, 'ntk')

In [13]:
kernel_train_m = onp.load('../kernel_vgg_4096.npy')

In [14]:
def model_fn(kernel_fn, x_train=None, x_test=None, fx_train_0=0., fx_test_0=0., t=None, ntk_train_train=None):
    # Kernel
    if ntk_train_train is None:
        ntk_train_train = kernel_fn(x_train, x_train, 'ntk')
    
    ntk_test_train = kernel_fn(x_test, x_train, 'ntk')
    # Prediction
    predict_fn = nt.predict.gradient_descent_mse(ntk_train_train, y_train, diag_reg=diag_reg) # no convariance
    
    return predict_fn(t, fx_train_0, fx_test_0, ntk_test_train) # fx_train_0, fx_test_0 = (0, 0) for infinite width

# loss

In [15]:
@jit
def l2_loss_v1(logits, labels, weighting=1):
    """
    Tensorflow version of L2 loss (without sqrt)
    """
    return np.sum(((logits - labels)**2) * weighting) / 2
    
@jit
def l2_loss_v2(logits, lables):
    """
    Normal L2 loss
    """
    return np.linalg.norm(logits - labels)

@jit
def cross_entropy_loss(logits, lables):
    return -np.sum(logsoftmax(logits) * lables)
    
@jit
def mse_loss(logits, lables):
    return 0.5 * np.mean((logits - lables) ** 2)

# attack algorithms

In [16]:
def fast_gradient_method(model_fn, kernel_fn, obj_fn, grads_fn, x_train=None, y_train=None, x_test=None, 
                         y=None, t=None, loss_weighting=None, fx_train_0=0., fx_test_0=0., eps=0.3, 
                         norm=np.inf, clip_min=None, clip_max=None, targeted=False):
    """
    JAX implementation of the Fast Gradient Method.
    :param model_fn: a callable that takes an input tensor and returns the model logits.
    :param x: input tensor.
    :param eps: epsilon (input variation parameter); see https://arxiv.org/abs/1412.6572.
    :param norm: Order of the norm (mimics NumPy). Possible values: np.inf or 2.
    :param clip_min: (optional) float. Minimum float value for adversarial example components.
    :param clip_max: (optional) float. Maximum float value for adversarial example components.
    :param y: (optional) Tensor with one-hot true labels. If targeted is true, then provide the
            target one-hot label. Otherwise, only provide this parameter if you'd like to use true
            labels when crafting adversarial samples. Otherwise, model predictions are used
            as labels to avoid the "label leaking" effect (explained in this paper:
            https://arxiv.org/abs/1611.01236). Default is None. This argument does not have
            to be a binary one-hot label (e.g., [0, 1, 0, 0]), it can be floating points values
            that sum up to 1 (e.g., [0.05, 0.85, 0.05, 0.05]).
    :param targeted: (optional) bool. Is the attack targeted or untargeted?
            Untargeted, the default, will try to make the label incorrect.
            Targeted will instead try to move in the direction of being more like y.
    :return: a tensor for the adversarial example
    """
    if norm not in [np.inf, 2]:
        raise ValueError("Norm order must be either np.inf or 2.")
        
    x = x_test
        
    # test independent
    if obj_fn == 'untargeted':
        grads = grads_fn(x_train, x_test, y_train, y, kernel_fn, t)
        
    else:
        raise ValueError("Objective function must be either train(ntk_train_train) or test(predict_fn)")

    axis = list(range(1, len(grads.shape)))
    eps_div = 1e-12
    
    if norm == np.inf:
        perturbation = eps * np.sign(grads)
    elif norm == 1:
        raise NotImplementedError("L_1 norm has not been implemented yet.")
    elif norm == 2:
        square = np.maximum(eps_div, np.sum(np.square(grads), axis=axis, keepdims=True))
        perturbation = grads / np.sqrt(square)
    
    # TODO
    adv_x = x + perturbation
    
    # If clipping is needed, reset all values outside of [clip_min, clip_max]
    if (clip_min is not None) or (clip_max is not None):
        # We don't currently support one-sided clipping
        assert clip_min is not None and clip_max is not None
        adv_x = np.clip(adv_x, a_min=clip_min, a_max=clip_max)
    
    return adv_x

In [17]:
def iter_fast_gradient_method(model_fn, kernel_fn, obj_fn, grads_fn, x_train=None, y_train=None,
                               x_test=None, y=None, t=None, loss_weighting=None, fx_train_0=0., fx_test_0=0., 
                               eps=0.3, eps_iter=0.03, nb_iter=10, norm=np.inf, clip_min=None, clip_max=None, 
                               targeted=False, rand_init=None, rand_minmax=0.3):
    """
    This class implements either the Basic Iterative Method
    (Kurakin et al. 2016) when rand_init is set to 0. or the
    Madry et al. (2017) method when rand_minmax is larger than 0.
    Paper link (Kurakin et al. 2016): https://arxiv.org/pdf/1607.02533.pdf
    Paper link (Madry et al. 2017): https://arxiv.org/pdf/1706.06083.pdf
    :param model_fn: a callable that takes an input tensor and returns the model logits.
    :param x: input tensor.
    :param eps: epsilon (input variation parameter); see https://arxiv.org/abs/1412.6572.
    :param eps_iter: step size for each attack iteration
    :param nb_iter: Number of attack iterations.
    :param norm: Order of the norm (mimics NumPy). Possible values: np.inf or 2.
    :param clip_min: (optional) float. Minimum float value for adversarial example components.
    :param clip_max: (optional) float. Maximum float value for adversarial example components.
    :param y: (optional) Tensor with true labels. If targeted is true, then provide the
            target label. Otherwise, only provide this parameter if you'd like to use true
            labels when crafting adversarial samples. Otherwise, model predictions are used
            as labels to avoid the "label leaking" effect (explained in this paper:
            https://arxiv.org/abs/1611.01236). Default is None.
    :param targeted: (optional) bool. Is the attack targeted or untargeted?
            Untargeted, the default, will try to make the label incorrect.
            Targeted will instead try to move in the direction of being more like y.
    :return: a tensor for the adversarial example
    """

    assert eps_iter <= eps, (eps_iter, eps)
    if norm == 1:
        raise NotImplementedError("It's not clear that FGM is a good inner loop"
                                  " step for PGD when norm=1, because norm=1 FGM "
                                  " changes only one pixel at a time. We need "
                                  " to rigorously test a strong norm=1 PGD "
                                  "before enabling this feature.")
    if norm not in [np.inf, 2]:
        raise ValueError("Norm order must be either np.inf or 2.")
        
    x = x_test
    
    # Initialize loop variables
    if rand_init:
        rand_minmax = eps
        eta = random.uniform(new_key, x.shape, minval=-rand_minmax, maxval=rand_minmax)
    else:
        eta = np.zeros_like(x)

    # Clip eta
    eta = clip_eta(eta, norm, eps)
    adv_x = x + eta
    if clip_min is not None or clip_max is not None:
        adv_x = np.clip(adv_x, a_min=clip_min, a_max=clip_max)
        
    for i in range(nb_iter):
        adv_x = fast_gradient_method(model_fn, kernel_fn, obj_fn, grads_fn, x_train, y_train, adv_x, 
                                        y, t, loss_weighting, fx_train_0, fx_test_0, eps_iter, norm, 
                                        clip_min, clip_max, targeted)

        # Clipping perturbation eta to norm norm ball
        eta = adv_x - x
        eta = clip_eta(eta, norm, eps)
        adv_x = x + eta

        # Redo the clipping.
        # FGM already did it, but subtracting and re-adding eta can add some
        # small numerical error.
        if clip_min is not None or clip_max is not None:
            adv_x = np.clip(adv_x, a_min=clip_min, a_max=clip_max)
    
    return adv_x

# Attack Hyperparameters

In [18]:
if DATASET == 'mnist':
    eps = 0.3
    eps_iter_10 = (eps/10)*1.1
    eps_iter_100 = (eps/100)*1.1
    eps_iter_1000 = (eps/1000)*1.1
    
elif DATASET == 'cifar10':
    eps = eps
    eps_iter_10 = (eps/10)*1.1
    # eps_iter_100 = (eps/100)*1.1
    eps_iter_100 = 1/255

In [19]:
def evaluate_accuracy(x_train, x_test, y_test, model_fn, kernel_fn, t=None, attack_type=None, ntk_train_train=None):
    
    y_train_predict, y_test_predict = model_fn(kernel_fn, x_train, x_test, 
                                               t=t, ntk_train_train=ntk_train_train)
    
    selected_table = correct(y_test_predict, y_test)
    print("Accuray({:s}): {:.2f}".format(attack_type, onp.mean(selected_table)))
    print("Loss({:s}): {:.4f}".format('cross entropy', cross_entropy_loss(y_test, y_test_predict)))
    print("Loss({:s}): {:.4f}".format('mse', mse_loss(y_test, y_test_predict)))
    return selected_table

In [20]:
def evaluate_robustness(x_train, x_test, y_test, model_fn, kernel_fn, selected_table, t=None, 
                        attack_type=None, ntk_train_train=None):
    
    y_train_predict, y_test_predict = model_fn(kernel_fn, x_train, x_test,
                                               t=t, ntk_train_train=ntk_train_train)
    
    y_test_predict = onp.asarray(y_test_predict)
    y_test_predict_select = y_test_predict[onp.asarray(selected_table)]
    y_test_select = y_test[onp.asarray(selected_table)]
    print("Robustness({:s}): {:.2f}".format(attack_type, onp.mean(correct(y_test_predict_select, y_test_select))))
    
    return

# adv_x generation

In [21]:
def inv(k):
        #inverse with diag_reg
        return onp.linalg.inv(k + diag_reg * onp.eye(k.shape[0]))

In [22]:
kernel_train_inv_m = inv(kernel_train_m)

In [23]:
def gen_adv_x(kernel_fn, x_test, y_test, t=None):
    
    kernel_fn = jit(kernel_fn, static_argnums=(2,))
    
    def inv(k):
        #inverse with diag_reg
        return onp.linalg.inv(k + diag_reg * onp.eye(k.shape[0]))
    
    ntk_train_train     = kernel_train_m
    ntk_train_train_inv = kernel_train_inv_m
    
    
    def test_loss_adv_mse(x_train, x_test, y_train, y, kernel_fn, t=None, diag_reg=diag_reg,
                          ntk_train_train=ntk_train_train):
        
        ntk_test_train = kernel_fn(x_test[None], x_train, 'ntk')

        predict_fn = nt.predict.gradient_descent_mse(ntk_train_train, y_train, diag_reg=diag_reg)
        # predict_fn(t, train_0, test_0, kernel_matrix)
        pred = predict_fn(t, 0., 0., ntk_test_train)[1]
        
        # loss = -mse_loss(pred, y)
        loss = -np.sum(logsoftmax(pred) * y)
        return loss
    
    test_mse_grads_fn = jit(vmap(grad(test_loss_adv_mse, argnums=1), in_axes=(None, 0, None, 0, None, None), 
                                 out_axes=0), static_argnums=(4,))
    
    #clean
    # print('evaluateing clean...')
    # selected_table = evaluate_accuracy(x_train, x_test, y_test, model_fn=model_fn,
    #                                    kernel_fn=kernel_fn, t=t, attack_type='Clean', ntk_train_train=ntk_train_train)

    # FGSM
    print('generating FGSM data...')
    adv_x_FGSM = fast_gradient_method(model_fn=model_fn, kernel_fn=kernel_fn, obj_fn='untargeted', 
                                      grads_fn=test_mse_grads_fn, x_train=x_train, y_train=y_train, 
                                      x_test=x_test, y=y_test, t=t, eps=eps, clip_min=0, clip_max=1)
    # print('evaluateing FGSM...')
    # evaluate_accuracy(x_train, adv_x_FGSM, y_test, model_fn=model_fn,
    #                     kernel_fn=kernel_fn, t=t, attack_type='FGSM', ntk_train_train=ntk_train_train)
    # 
    # evaluate_robustness(x_train, adv_x_FGSM, y_test, model_fn=model_fn,
    #                     kernel_fn=kernel_fn, selected_table=selected_table, t=t, 
    #                     attack_type='FGSM', ntk_train_train=ntk_train_train)

    # I-FGSM 100
    print('generating I-FGSM-100 data...')
    adv_x_IFGSM_100 = iter_fast_gradient_method(model_fn=model_fn, kernel_fn=kernel_fn, obj_fn='untargeted', 
                                                 grads_fn=test_mse_grads_fn, x_train=x_train, 
                                                 y_train=y_train, x_test=x_test, y=y_test, t=t, 
                                                 eps=eps, eps_iter=eps_iter_100, nb_iter=100, 
                                                 clip_min=0, clip_max=1, rand_init=None, rand_minmax=eps)
    # print('evaluateing I-FGSM-100...')
    # evaluate_accuracy(x_train, adv_x_IFGSM_100, y_test, model_fn=model_fn,
    #                     kernel_fn=kernel_fn, t=t, attack_type='I-FGSM-100', ntk_train_train=ntk_train_train)
    # 
    # evaluate_robustness(x_train, adv_x_IFGSM_100, y_test, model_fn=model_fn,
    #                     kernel_fn=kernel_fn, selected_table=selected_table, t=t, 
    #                     attack_type='I-FGSM-100', ntk_train_train=ntk_train_train)

    return adv_x_FGSM, adv_x_IFGSM_100

In [24]:
adv_x_FGSM, adv_x_IFGSM_100 = [], []
for batch_id in range(test_size//test_batch_size):
    print("batch number %d"%(batch_id))
    fgsm, ifgsm = gen_adv_x(kernel_fn, 
                            x_test[batch_id*test_batch_size:(batch_id+1)*test_batch_size], 
                            y_test[batch_id*test_batch_size:(batch_id+1)*test_batch_size],
                            time)
    adv_x_FGSM.append(fgsm)
    adv_x_IFGSM_100.append(ifgsm)
    print("-------------------------")

batch number 0
generating FGSM data...
generating I-FGSM-100 data...
-------------------------
batch number 1
generating FGSM data...
generating I-FGSM-100 data...
-------------------------
batch number 2
generating FGSM data...
generating I-FGSM-100 data...
-------------------------
batch number 3
generating FGSM data...
generating I-FGSM-100 data...
-------------------------


In [25]:
FGSM = onp.concatenate(adv_x_FGSM, axis=0)
IFGSM_100 = onp.concatenate(adv_x_IFGSM_100, axis=0)

onp.save("cifar-FGSM-eps=%s-t=%s.npy"%(eps, time), FGSM)
onp.save("cifar-IFGSM-eps=%s-t=%s.npy"%(eps, time), IFGSM_100)

In [26]:
print('evaluateing clean...')
selected_table = evaluate_accuracy(x_train, x_test, y_test, model_fn=model_fn,
                                   kernel_fn=kernel_fn, t=time, attack_type='Clean', ntk_train_train=kernel_train_m)
print("-------------------------")
print('evaluateing FGSM...')
evaluate_accuracy(x_train, FGSM, y_test, model_fn=model_fn,
                    kernel_fn=kernel_fn, t=time, attack_type='FGSM', ntk_train_train=kernel_train_m)

evaluate_robustness(x_train, FGSM, y_test, model_fn=model_fn,
                    kernel_fn=kernel_fn, selected_table=selected_table, t=time, 
                    attack_type='FGSM', ntk_train_train=kernel_train_m)
print("-------------------------")
print('evaluateing I-FGSM-100...')
evaluate_accuracy(x_train, IFGSM_100, y_test, model_fn=model_fn,
                    kernel_fn=kernel_fn, t=time, attack_type='I-FGSM-100', ntk_train_train=kernel_train_m)

evaluate_robustness(x_train, IFGSM_100, y_test, model_fn=model_fn,
                    kernel_fn=kernel_fn, selected_table=selected_table, t=time, 
                    attack_type='I-FGSM-100', ntk_train_train=kernel_train_m)

evaluateing clean...
Accuray(Clean): 0.50
Loss(cross entropy): 136.8915
Loss(mse): 0.0332
-------------------------
evaluateing FGSM...
Accuray(FGSM): 0.00
Loss(cross entropy): 173.2489
Loss(mse): 0.0943
Robustness(FGSM): 0.00
-------------------------
evaluateing I-FGSM-100...
Accuray(I-FGSM-100): 0.00
Loss(cross entropy): 176.7436
Loss(mse): 0.1056
Robustness(I-FGSM-100): 0.00
